# 🔤 Byte Pair Encoding (BPE) Implementation

**Byte Pair Encoding (BPE)** is a tokenization algorithm used by many modern language models to convert text into tokens. It iteratively merges the most frequent pair of bytes or characters in the text.

## 🎯 What You'll Learn

1. Understanding BPE tokenization process
2. Converting text to bytes and tokens
3. Finding frequent character pairs
4. Merging pairs to build vocabulary
5. Encoding and decoding with BPE

---

## 📊 Project Overview

**Goal:** Implement a simple BPE tokenizer from scratch.

**Process:**
1. 📝 Start with raw text
2. 🔢 Convert to bytes
3. 📈 Find most frequent pairs
4. 🔄 Merge pairs iteratively
5. 🗂️ Build vocabulary

---

## 🚀 Section 1: Initial Setup


In [ ]:
MY_STRING = "Hello there, running fastest to the tallest hills."

---

## 🔢 Section 2: Converting Text to Bytes

BPE starts by converting text to bytes (UTF-8 encoding), which gives us a list of integers (0-255) representing each character.


In [ ]:
raw_bytes = MY_STRING.encode('utf-8')
raw_bytes

---

## 📊 Section 3: Building Token List

Now we convert the bytes to a list of integers - this is our initial token sequence where each byte is a token.


In [ ]:
tokens = list(map(int, raw_bytes))
tokens

---

## 🔍 Section 4: Finding Frequent Pairs

The core of BPE is finding the most frequent consecutive pair of tokens. We'll count all adjacent pairs and identify which pair appears most often.


In [ ]:
def get_stats(tokens: list[int]) -> dict[tuple[int, int], int]:
    """Count frequency of adjacent token pairs."""
    stats = {}
    for tok1, tok2 in zip(tokens, tokens[1:]):
        if (tok1, tok2) in stats:
            stats[(tok1, tok2)] += 1
        else:
            stats[(tok1, tok2)] = 1
    return stats

stats = get_stats(tokens)
top_pair = max(stats, key=stats.get)
print(f"Most frequent pair: {top_pair} (appears {stats[top_pair]} times)")
print(f"As characters: '{chr(top_pair[0])}{chr(top_pair[1])}'")
print(f"\nTop 5 pairs:")
for pair, count in sorted(stats.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  {pair} -> '{chr(pair[0])}{chr(pair[1])}': {count} times")

---

## 🔄 Section 5: Merging Token Pairs

Once we find the most frequent pair, we merge it into a single new token.

This function replaces all occurrences of the pair with a new token ID.


### 🔍 How the Merge Function Works

The `merge()` function scans through the token list and performs the following steps:

**Input Parameters:**
- `tokens`: List of current token IDs (e.g., `[72, 101, 108, 108, 111]`)
- `pair`: The pair to find and merge (e.g., `(108, 108)` representing "ll")
- `new_token_id`: The new ID to replace the pair (e.g., `256`)

**Algorithm:**
1. **Iterate** through tokens using index `i`
2. **Check** if current position has the matching pair:
   - If `tokens[i]` matches `pair[0]` AND `tokens[i+1]` matches `pair[1]`
   - Then: Add `new_token_id` to result and skip both tokens (`i += 2`)
3. **Otherwise**: Copy the current token unchanged and move to next (`i += 1`)
4. **Return** the new token list with merged pairs

**Example:**
- Input: `[72, 101, 108, 108, 111]` (represents "Hello")
- Pair to merge: `(108, 108)` (represents "ll")
- New token ID: `256`
- Output: `[72, 101, 256, 111]` (now "He[ll]o" where `[ll]` is token 256)

**Why Token 256?**
- Tokens 0-255 are reserved for individual bytes (UTF-8 encoding)
- New merged tokens start from 256 onwards
- Each merge creates a new vocabulary entry


In [ ]:
def merge(tokens: list[int], pair: tuple[int, int], new_token_id: int) -> list[int]:
    """Merge all occurrences of a pair into a single new token."""
    new_tokens = []
    i = 0
    while i < len(tokens):
        if i < len(tokens) - 1 and tokens[i] == pair[0] and tokens[i + 1] == pair[1]:
            new_tokens.append(new_token_id)
            i += 2
        else:
            new_tokens.append(tokens[i])
            i += 1
    return new_tokens

# Test the merge function
new_token_id = 256
merged_tokens = merge(tokens, top_pair, new_token_id)
print(f"Original token count: {len(tokens)}")
print(f"After merging: {len(merged_tokens)}")
print(f"Tokens saved: {len(tokens) - len(merged_tokens)}")


---

## 🔁 Section 6: Iterative BPE Training

Now we'll implement the full BPE training loop that repeatedly finds and merges the most frequent pairs until we reach our desired vocabulary size.


In [ ]:
def train_bpe(text: str, num_merges: int) -> tuple[dict[int, tuple[int, int]], list[int]]:
    """
    Train BPE tokenizer by performing specified number of merges.

    Returns:
        merges: Dictionary mapping new token IDs to the pair they represent
        tokens: Final token sequence after all merges
    """
    tokens = list(map(int, text.encode('utf-8')))
    merges = {}

    for i in range(num_merges):
        stats = get_stats(tokens)
        if not stats:
            break

        top_pair = max(stats, key=stats.get)
        new_token_id = 256 + i
        tokens = merge(tokens, top_pair, new_token_id)
        merges[new_token_id] = top_pair

        if (i + 1) % 5 == 0 or i < 5:
            print(f"Merge {i+1}: {top_pair} -> {new_token_id} (freq: {stats[top_pair]})")

    return merges, tokens

# Train BPE with 20 merges
num_merges = 5
merges, final_tokens = train_bpe(MY_STRING, num_merges)
print(f"\nFinal token count: {len(final_tokens)} (original: {len(list(MY_STRING.encode('utf-8')))})")
print(f"Compression ratio: {len(final_tokens) / len(list(MY_STRING.encode('utf-8'))):.2%}")


---

## 🗂️ Section 7: Building Vocabulary

Let's examine the vocabulary we've built. Each merge creates a new token that represents a pair of existing tokens.


In [ ]:
def decode_token(token_id: int, merges: dict[int, tuple[int, int]]) -> bytes:
    """Recursively decode a token ID back to bytes."""
    if token_id < 256:
        return bytes([token_id])

    pair = merges[token_id]
    return decode_token(pair[0], merges) + decode_token(pair[1], merges)

print("Vocabulary (new tokens):")
print("=" * 60)
for token_id, pair in merges.items():
    decoded = decode_token(token_id, merges)
    print(f"Token {token_id}: {pair} -> '{decoded.decode('utf-8', errors='replace')}'")


---

## 🔓 Section 8: Decoding Tokens Back to Text

Now let's decode our tokenized sequence back to the original text to verify our implementation works correctly.


In [ ]:
def decode(tokens: list[int], merges: dict[int, tuple[int, int]]) -> str:
    """Decode a sequence of tokens back to text."""
    byte_sequence = b""
    for token in tokens:
        byte_sequence += decode_token(token, merges)
    return byte_sequence.decode('utf-8', errors='replace')

decoded_text = decode(final_tokens, merges)
print("Original text:")
print(f"  '{MY_STRING}'")
print(f"\n\nTokenized text: {final_tokens} \n")
print("\nDecoded text:")
print(f"  '{decoded_text}'")
print(f"\nMatch: {MY_STRING == decoded_text}")


---

## 🔐 Section 9: Encoding New Text

Let's create an encode function that can tokenize new text using our trained BPE vocabulary.


In [ ]:
def encode(text: str, merges: dict[int, tuple[int, int]]) -> list[int]:
    """Encode text using the trained BPE merges."""
    tokens = list(map(int, text.encode('utf-8')))

    while len(tokens) >= 2:
        stats = get_stats(tokens)

        # Find the pair that should be merged based on our learned merges
        pair_to_merge = None
        for token_id in sorted(merges.keys()):
            pair = merges[token_id]
            if pair in stats:
                pair_to_merge = (pair, token_id)
                break

        if pair_to_merge is None:
            break

        pair, token_id = pair_to_merge
        tokens = merge(tokens, pair, token_id)

    return tokens

# Test encoding
test_text = "Hello there"
encoded = encode(test_text, merges)
decoded_back = decode(encoded, merges)

print(f"Test text: '{test_text}'")
print(f"Encoded tokens: {encoded}")
print(f"Token count: {len(encoded)}")
print(f"Decoded: '{decoded_back}'")
print(f"Match: {test_text == decoded_back}")


---

## 📊 Section 10: Visualizing Tokenization

Let's visualize how different texts are tokenized with our BPE vocabulary.


In [ ]:
test_strings = [
    "Hello there",
    "running fastest",
    "tallest hills",
    "Hello world",
    "testing BPE"
]

print("Tokenization comparison:")
print("=" * 70)
for text in test_strings:
    encoded = encode(text, merges)
    byte_count = len(text.encode('utf-8'))
    print(f"\nText: '{text}'")
    print(f"  Bytes: {byte_count}")
    print(f"  Tokens: {len(encoded)}")
    print(f"  Compression: {len(encoded)/byte_count:.2%}")
    print(f"  Token IDs: {encoded}")


---

## 📚 Summary

### ✨ Key Concepts Covered

1. **Byte Representation**: Converting text to bytes as the foundation of BPE
2. **Pair Frequency Analysis**: Counting adjacent token pairs to find patterns
3. **Iterative Merging**: Building vocabulary by repeatedly merging frequent pairs
4. **Token Vocabulary**: Creating new tokens that represent character sequences
5. **Encoding**: Converting text to tokens using learned merges
6. **Decoding**: Reconstructing original text from tokens
7. **Compression**: Reducing token count while preserving information

### 💡 Key Insights

- ✅ **BPE is data-driven**: The vocabulary adapts to patterns in the training text
- ✅ **Hierarchical encoding**: New tokens build on previous merges
- ✅ **Compression vs expressiveness**: More merges = smaller sequences but larger vocabulary
- ✅ **Subword tokenization**: BPE naturally handles rare words by breaking them into subwords
- ✅ **Deterministic**: Same training data and merge count produces same vocabulary

### 🎯 Real-World Applications

BPE is used in many modern language models:
- 🔹 **GPT models**: Use BPE tokenization
- 🔹 **BERT variants**: Many use WordPiece (similar to BPE)
- 🔹 **Translation models**: BPE helps handle rare words across languages
- 🔹 **Code models**: BPE adapts to programming syntax patterns

### 📖 Next Steps

- 🔹 Train BPE on larger datasets
- 🔹 Experiment with different vocabulary sizes
- 🔹 Compare BPE with other tokenization methods
- 🔹 Implement BPE variants (e.g., byte-level BPE)
- 🔹 Explore tokenization in production models

---

### 🎓 Congratulations!

You now understand how Byte Pair Encoding works and can implement a basic BPE tokenizer from scratch!
